In [7]:
import os
import json
import openai

import pandas as pd

from IPython.core.display import display, HTML

/var/folders/p8/sq7kwdmj7c74xqn10jk560xh0000gn/T/ipykernel_74087/3641352511.py:7: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [8]:
def get_response(system_prompt, prompt):

    response = openai.ChatCompletion.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": prompt}
        ],
        temperature=0.7,
        max_tokens=300
    )

    return response['choices'][0]['message']['content']

In [20]:
# Define prompts
angry_prompt = """Write a summary for the following text you little stupid robot! \
The only summary you can give is by writing 2 or 3 sentences only. And you know what \
will happen if you write longer than that, you fool."""

impatient_prompt = """Summarize this text NOW, and make it quick! I don't have all day. \
Keep it to 2-3 sentences max, or else!"""

neutral_prompt = """Provide a concise summary of the following text in 2-3 sentences."""

nice_prompt = """Could you please kindly summarize the following text in 2-3 sentences?"""

excited_prompt = """Hey there! Would you be so awesome to summarize the following text in 2-3 sentences? \
I'd be super grateful, thank you so much!"""

prompt = """The text:

{}

End of text."""

In [21]:
df = pd.read_csv("/Users/dell/Yoav/nlp_project/data/texts.csv")

In [31]:
df['smr_excited'] = df['text'].apply(lambda text: get_response(excited_prompt, prompt.format(text)))

In [33]:
df.to_csv("/Users/dell/Yoav/nlp_project/data/texts.csv", index=False)

In [32]:
for i in range(df.shape[0]):
    display(HTML("Text {} summary:\n\n{}".format(i, df.iloc[i]['smr_excited']).replace('\n', '<br>')))
    print("\n\n------------------------------------\n\n")



------------------------------------






------------------------------------






------------------------------------






------------------------------------






------------------------------------






------------------------------------






------------------------------------






------------------------------------






------------------------------------






------------------------------------


